In [28]:
import sys
import os
import re
from query_rdfs import *

sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))
current_dir = os.path.dirname("__file__")
current_dir = current_dir if current_dir is not '' else '.'
# directory to scan for any txt files
data_dir_path = current_dir + '/rule'


known = {"height": 170, "weight": 55, "gender": "nam", 
         "age": 17, "PhysicalActiveLevel": "Hoạt động vừa", "target": "giữ cân"}
data_rule = []
function = []

In [29]:
with open(data_dir_path + "/rule.txt", "r", encoding="utf8") as f:
    for line in f:
        data_rule.append(line)
with open(data_dir_path + "/function.txt", "r", encoding="utf8") as f:
    for line in f:
        function.append(line)

In [4]:
print(function)

['\ufeffbmr,nu=655+(9.6*weight)+(1.8*height)-(4.7*age)\n', 'bmr,nam=66+(13.75 *weight)+(5*height)-(6.8*age)\n', 'bmi=weight/(height*height)\n', 'calo=BMRvalue*PhysicalActiveValue\n', '1.2:Quá ít hoạt động\n', '1.375:Ít hoạt động\n', '1.55:Hoạt động vừa\n', '1.725:Hoạt động nhiều\n', '1.9:Hoạt động cường độ cao']


# BMI function

In [5]:
def BMIvalueCalc(height, weight):
    height/=100
    return round(eval(function[2].split("=")[1]),2)

def BMIrangeCalc(bmi_value):
    return BMI_range_calc(bmi_value)

def BMIlevelCalc(bmi_range):
    return has_value(bmi_range)["value"]

# BMR function

In [6]:
def BMRvalueOfMaleCalc(height, weight, age):
    function[1] = function[1].split("=")[1]
    return round(eval(function[1]), 2)

def BMRvalueOfFemaleCalc(height, weight, age):
    function[1] = function[1].split("=")[1]
    return round(eval(function[1]), 2)

# print (BMIvalueCalc(height,weight)) 

# PA function

In [7]:
def PAvalueCalc(PhysicalActiveLevel):
    for count in range(4,9):
        result = function[count].split(":")
        if result[1].endswith("\n"):
            result[1] = result[1][:-1]
        if  result[1] == PhysicalActiveLevel:
            return float(result[0])

# CaloCalc function

In [21]:
def CaloCalc(BMRvalue, PhysicalActiveValue, target):
    result = round(eval(function[3].split("=")[1]),2)
    if target == "tăng cân":
        result +=500
    elif target == "giảm cân":
        result -=500
    return result

def CaloToKeepWeightCalc(calo_per_day):
    # breakfast: 16%, #Lunch: 40%, # Dinner: 34%
    return [round(calo_per_day*16/100, 2), 
            round(calo_per_day*40/100, 2), round(calo_per_day*34/100, 2)]    

In [9]:
def extract_rule_left(rule_left):
    rule_left = re.findall(r'\[(.*?)\]',rule_left)
    flag = True
    for content in rule_left:
        print(content)
        content = content.split(",")
        if content[0] == "1":
            if content[1] not in known:
                flag = False
                break
        elif content[0] == "2":
            try:
                if known[content[1]] != content[2]:
                    flag = False
                    break
            except:
                flag = False
                break
    return flag
                
def extract_rule_right(rule_right):
    rule_right = rule_right.split(",", 1)
    flag = True
    rule_right[0] = rule_right[0][1:]
    rule_right[1] = rule_right[1][:-2]
    print(rule_right)
    if rule_right[0] in known:
        flag = False
    else:
        funct = rule_right[1].split(':')[1]
        print(funct)
        known[rule_right[0]] = eval(funct)
    return flag
    
def extract_rule(rule):
    flag = True
    rule_left, rule_right = rule.split("|")
    flag = extract_rule_left(rule_left)
    if flag is False:
        return flag
    else: 
        flag = extract_rule_right(rule_right)
    return flag

In [32]:
count = 0
while count <= 9:
    if extract_rule(data_rule[count]) == True:
        count = 0
    else: count+=1        

1,height
1,weight
['BMIvalue', 'funct:BMIvalueCalc(known["height"],known["weight"])']
BMIvalueCalc(known["height"],known["weight"])
1,height
1,weight
['BMIvalue', 'funct:BMIvalueCalc(known["height"],known["weight"])']
1,BMIvalue
['BMIrange', 'funct:BMIrangeCalc(known["BMIvalue"])']
BMIrangeCalc(known["BMIvalue"])
1,height
1,weight
['BMIvalue', 'funct:BMIvalueCalc(known["height"],known["weight"])']
1,BMIvalue
['BMIrange', 'funct:BMIrangeCalc(known["BMIvalue"])']
1,BMIrange
['BMIlevel', 'funct:BMIlevelCalc(known["BMIrange"])']
BMIlevelCalc(known["BMIrange"])
1,height
1,weight
['BMIvalue', 'funct:BMIvalueCalc(known["height"],known["weight"])']
1,BMIvalue
['BMIrange', 'funct:BMIrangeCalc(known["BMIvalue"])']
1,BMIrange
['BMIlevel', 'funct:BMIlevelCalc(known["BMIrange"])']
2,gender,nam
1,height
1,weight
1,age
['BMRvalue', 'funct:BMRvalueOfMaleCalc(known["height"], known["weight"], known["age"])']
BMRvalueOfMaleCalc(known["height"], known["weight"], known["age"])
1,height
1,weight
['BMIvalue

In [26]:
print(extract_rule(data_rule[7]))

2,target,giữ cân
1,CaloPerDay
['CaloForEachMeal', 'funct:CaloToKeepWeightCalc(known["CaloPerDay"])']
CaloToKeepWeightCalc(known["CaloPerDay"])
True


In [33]:
print(known)

{'height': 170, 'weight': 55, 'gender': 'nam', 'age': 17, 'PhysicalActiveLevel': 'Hoạt động vừa', 'target': 'giữ cân', 'BMIvalue': 19.03, 'BMIrange': '18.50-24.99', 'BMIlevel': 'Bình_thường', 'BMRvalue': 1556.65, 'PhysicalActiveValue': 1.55, 'CaloPerDay': 2412.81, 'CaloForEachMeal': [386.05, 965.12, 820.36]}
